In [1]:
%matplotlib qt
import scipy.stats as st
import numpy as np
from scipy import fftpack
import scipy.ndimage.filters
import matplotlib.pyplot as plt
import skimage.io
import skimage.exposure
import skimage.util
import skimage.morphology
import scipy.ndimage
from skimage.measure import compare_ssim
import cv2

In [2]:
filePath = './strb1.jpg';

In [3]:
img = cv2.imread(filePath)
cv2.imwrite('1_out_initial.jpg', img)

True

In [4]:
# Isolate very bright red colors
redBoundary = ([0, 0, 100], [200, 200, 255])
mask§ = cv2.inRange(img, np.array(redBoundary[0],dtype = "uint8"), np.array(redBoundary[1], dtype = "uint8"))
outputRed = cv2.bitwise_and(img, img, mask = mask)

greenBoundary = ([0, 100, 0], [200, 255, 200])
mask = cv2.inRange(img, np.array(greenBoundary[0],dtype = "uint8"), np.array(greenBoundary[1], dtype = "uint8"))
outputGreen = cv2.bitwise_and(img, img, mask = mask)

blueBoundary = ([100, 0, 0], [200, 255, 200])
mask = cv2.inRange(img, np.array(blueBoundary[0],dtype = "uint8"), np.array(blueBoundary[1], dtype = "uint8"))
outputBlue = cv2.bitwise_and(img, img, mask = mask)

cv2.imwrite("2_red_green_blue.jpg", np.hstack([outputRed, outputGreen, outputBlue]))
cv2.imwrite("2_reconstructed.jpg", cv2.add(outputRed, cv2.add(outputGreen, outputBlue)))
onlyRedImage =  np.int32(outputRed) -  np.int32(outputGreen) -  np.int32(outputBlue)
cv2.imwrite('2_only_red.jpg', onlyRedImage)

True

In [31]:
# Blur the red image
ksize = (3, 3) 
blurredRedImage = cv2.blur(onlyRedImage, ksize, cv2.BORDER_DEFAULT) 
cv2.imwrite('3_blurred_red.jpg', blurredRedImage)


True

In [63]:
# Remove almost black pixels
blackBoundary = ([0, 0, 0], [20, 20, 20])
mask = cv2.inRange(img, np.array(blackBoundary[0],dtype = "uint8"), np.array(blackBoundary[1], dtype = "uint8"))
mask = 255 - mask
outputNoBlack = cv2.bitwise_and(blurredRedImage, blurredRedImage, mask = mask)

print(mask)

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [64]:
# Make image gray scaled
imgGrey = cv2.cvtColor(np.array(outputNoBlack, dtype = "uint8"), cv2.COLOR_BGR2GRAY)
cv2.imwrite('4_grayscale.jpg', imgGrey)

True

In [29]:
threshold, otsuImage = cv2.threshold(imgGrey, 0, 255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
cv2.imwrite('5_otsu.jpg', otsuImage)

True

In [8]:
# Erode the image
kernel = np.ones((5,5),np.uint8)
erodedRedImage = cv2.erode(blurredRedImage, kernel, iterations = 1)  
cv2.imwrite('4_eroded_red.jpg', erodedRedImage)

error: OpenCV(4.2.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/morph.simd.hpp:756: error: (-213:The function/feature is not implemented) Unsupported data type (=20) in function 'getMorphologyRowFilter'
